# hello world

fix error by retry


In [ ]:
import random
from tenacity import retry


@retry
def do_something_unreliable():
    if random.randint(0, 10) > 1:
        raise IOError("Broken sauce, everything is hosed!!!111one")
    else:
        return "Awesome sauce!"


print(do_something_unreliable())

# 指定停止条件


## 重试次数


In [ ]:
from tenacity import stop_after_attempt


@retry(stop=stop_after_attempt(5))
def stop_after_5_attempts():
    print("Stopping after 5 attempts")
    raise Exception


stop_after_5_attempts()

## 重试几秒


In [ ]:
from tenacity import stop_after_delay


@retry(stop=stop_after_delay(2))
def stop_after_2_s():
    print("Stopping after 10 seconds")
    raise Exception


stop_after_2_s()

# 指定重试前等待


## 等待几秒


In [ ]:
from tenacity import wait_fixed


@retry(wait=wait_fixed(2))
def wait_2_s():
    print("Wait 2 second between retries")
    raise Exception


wait_2_s()

## 等待随机时间


In [ ]:
from tenacity import wait_random


@retry(wait=wait_random(min=1, max=2))
def wait_random_1_to_2_s():
    print("Randomly wait 1 to 2 seconds between retries")
    raise Exception


wait_random_1_to_2_s()

## 固定+随机等待时间


In [ ]:
from tenacity import wait_fixed
from tenacity import wait_random


@retry(wait=wait_fixed(3) + wait_random(0, 2))
def wait_fixed_jitter():
    print("Wait at least 3 seconds, and add up to 2 seconds of random delay")
    raise Exception


wait_fixed_jitter()

## 等待时间-指数式递增


In [ ]:
from tenacity import wait_exponential

# 最短等待时间为1，最小等待4秒，每次增加1倍，最多10
@retry(wait=wait_exponential(multiplier=1, min=4, max=10))
def wait_exponential_1():
    print(
        "Wait 2^x * 1 second between each retry starting with 4 seconds, then up to 10 seconds, then 10 seconds afterwards"
    )
    raise Exception


wait_exponential_1()

## 等待时间-指随机数式递增


In [ ]:
from tenacity import wait_random_exponential


@retry(wait=wait_random_exponential(multiplier=1, max=60))
def wait_exponential_jitter():
    print(
        "Randomly wait up to 2^x * 1 seconds between each retry until the range reaches 60 seconds, then randomly up to 60 seconds afterwards"
    )
    raise Exception


wait_exponential_jitter()

# 指定异常重试


In [ ]:
from tenacity import retry_if_exception_type


@retry(retry=retry_if_exception_type(IOError))
def might_io_error():
    print("Retry forever with no wait if an IOError occurs, raise any other errors")
    raise Exception


might_io_error()

# 指定返回值重试


In [ ]:
from tenacity import retry_if_result


def is_none_p(value):
    """Return True if value is None"""
    return value is None


@retry(retry=retry_if_result(is_none_p))
def might_return_none():
    print("Retry with no wait if return value is None")


might_return_none()

# 混合重试条件


## 或


In [ ]:
from tenacity import stop_after_attempt
from tenacity import stop_after_delay


@retry(stop=(stop_after_delay(1) | stop_after_attempt(5)))
def stop_after_1_s_or_5_retries():
    print("Stopping after 1 seconds or 5 retries")
    raise Exception

## 链式


In [ ]:
from tenacity import wait_chain
from tenacity import wait_fixed


@retry(
    wait=wait_chain(
        *[wait_fixed(3) for i in range(3)]
        + [wait_fixed(7) for i in range(2)]
        + [wait_fixed(9)]
    )
)
def wait_fixed_chained():
    print(
        "Wait 3s for 3 attempts, 7s for the next 2 attempts and 9s for all attempts thereafter"
    )
    raise Exception


wait_fixed_chained()

# 手动引发重试


In [ ]:
from tenacity import TryAgain


@retry
def do_something():
    result = 23
    if result == 23:
        print("retry")
        raise TryAgain


do_something()

# 异常处理


## 抛出原异常

默认会吞掉原异常


In [ ]:
from tenacity import stop_after_attempt


@retry(reraise=True, stop=stop_after_attempt(3))
def raise_my_exception():
    raise ValueError("Fail")


try:
    raise_my_exception()
except ValueError:
    pass

raise_my_exception()

## 异常回调


### 记录日志-after 异常


In [44]:
import logging
import sys

from tenacity import after_log

logging.basicConfig(stream=sys.stderr, level=logging.DEBUG)

logger = logging.getLogger(__name__)


@retry(stop=stop_after_attempt(3), after=after_log(logger, logging.DEBUG))
def raise_my_exception():
    raise ValueError("Fail")


try:
    raise_my_exception()
except Exception:
    pass

DEBUG:__main__:Finished call to '__main__.raise_my_exception' after 0.000(s), this was the 1st time calling it.
DEBUG:__main__:Finished call to '__main__.raise_my_exception' after 0.001(s), this was the 2nd time calling it.
DEBUG:__main__:Finished call to '__main__.raise_my_exception' after 0.002(s), this was the 3rd time calling it.


### 记录异常-before 重新回调


In [43]:
import logging
import sys

from tenacity import before_log

logging.basicConfig(stream=sys.stderr, level=logging.DEBUG)
logger = logging.getLogger(__name__)


def before_call():
    print("before_call exec")


@retry(stop=stop_after_attempt(3), before=before_log(logger, logging.DEBUG))
def raise_my_exception():
    raise ValueError("Fail")


try:
    raise_my_exception()

except Exception:
    pass

DEBUG:__main__:Starting call to '__main__.raise_my_exception', this is the 1st time calling it.
DEBUG:__main__:Starting call to '__main__.raise_my_exception', this is the 2nd time calling it.
DEBUG:__main__:Starting call to '__main__.raise_my_exception', this is the 3rd time calling it.


### 自定义调


In [ ]:
def my_error_call(retry_state):
    print(f"my_error_call exec param is {retry_state}")
    return retry_state.outcome.result()


@retry(stop=stop_after_attempt(3), retry_error_callback=my_error_call)
def raise_my_exception():
    raise ValueError("Fail")


raise_my_exception()

# 查看重试次数


In [ ]:
@retry(stop=stop_after_attempt(3))
def raise_my_exception():
    raise ValueError("Fail")


try:
    raise_my_exception()
except Exception:
    pass

print(raise_my_exception.retry.statistics)